In [2]:
import pandas as pd



In [3]:
'''
quota = pd.read_html('quota.html', match= "Quota")
print(len(quota))
df = quota[0]
df.head()
df.columns = ['Department', 'Status', "Quota", "Current","a"]
df.head()
df.drop("a",axis=1, inplace=True)
df
df.drop(labels=range(0,2), inplace=True)
df = df.reset_index(drop=True)
df
'''

'''
import requests
from bs4 import BeautifulSoup
 
 
url = 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=MIS&bolum=MANAGEMENT+INFORMATION+SYSTEMS'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
urls = []
for link in soup.find_all('a'):
    print(link.get('href'))
'''

"\nimport requests\nfrom bs4 import BeautifulSoup\n \n \nurl = 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=MIS&bolum=MANAGEMENT+INFORMATION+SYSTEMS'\nreqs = requests.get(url)\nsoup = BeautifulSoup(reqs.text, 'html.parser')\n \nurls = []\nfor link in soup.find_all('a'):\n    print(link.get('href'))\n"

In [4]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver1 = webdriver.Chrome(options=options)
driver1.get("https://registration.boun.edu.tr/buis/General/schedule.aspx?p=semester")

wait = WebDriverWait(driver1, 10)
element1 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="submit"]')))
element_id1 = element1.get_attribute('id')
go_button = wait.until(EC.presence_of_element_located((By.ID, element_id1)))


go_button.click()

driver1.implicitly_wait(3)

link_elements = driver1.find_elements(By.TAG_NAME, 'a')

urls = []

for link in link_elements:
    urls.append(link.get_attribute('href'))

driver1.quit()


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np
import tkinter as tk
from tkinter import ttk
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

def get_data():
    dfs = []

    for url in urls:
        driver.get(url)
        info_links = driver.find_elements(By.LINK_TEXT,"Info")
        for info_link in info_links:
            info_link.click()
            # Switch to the new window
            new_window = driver.window_handles[-1]
            driver.switch_to.window(new_window)
            try:
                # Find the table element
                table = driver.find_element(By.TAG_NAME, "table")
                # Convert the table content to a DataFrame
                df2 = pd.read_html(table.get_attribute('outerHTML'))[0]
                strongs = driver.find_elements(By.TAG_NAME, "strong")
                strongs_text = [strong.text for strong in strongs]
                strongs_text_repeated = np.repeat(strongs_text, len(df2))
                df2["Code"] = strongs_text_repeated[:len(df2)]
                dfs.append(df2)
            except IndexError:
                # No table found, move on to the next link
                pass
            finally:
                # Close the new window
                driver.close()
                # Switch back to the main window
                main_window = driver.window_handles[0]
                driver.switch_to.window(main_window)


    result = pd.concat(dfs, ignore_index=True)
    result = result[["Code",0,1,2,3,4]]
    result.drop(result.columns[[5]], axis=1, inplace=True)
    result.columns = ['Code', 'Department', 'Status', 'Quota', 'Current Quota']

    words = ["Departmental Quotas:", "Department", "OZELLISANS", "PHD","MASTER","OZELMASTER","OZELPHD"]

    result = result[result.Department.isin(words) == False]
    result = result[result.Status.isin(words) == False]
    result.reset_index(drop=True, inplace=True)

    # result.to_csv('data.csv', index=False)
    # testdata = pd.read_csv("data.csv")

    def get_availability(row):
        quota = row['Quota']
        try:
            quota = int(quota)
        except ValueError:
            pass
        current_quota = int(row['Current Quota'])

            
        if isinstance(quota, int) and current_quota == quota:
            return "Full"
        elif isinstance(quota, int) and current_quota < quota:
            return "Available"
        elif isinstance(quota, int) and current_quota > quota:
            return "Try to get consent"
        elif isinstance(quota, str):
            return "Available"
        

    result["Availability"] = result.apply(get_availability, axis=1)
    return result




In [11]:
table_data = get_data()
table_data.to_csv("data1.csv", index=False)

In [20]:
import pandas as pd
import tkinter as tk
from tkinter import ttk

testingdata = pd.read_csv("data1.csv")

def update_table(code_filter='', availability_filter=''):
    filtered_data = testingdata.copy()
    if code_filter:
        filtered_data = filtered_data[filtered_data['Code'].str.contains(code_filter)]
    if availability_filter:
        filtered_data = filtered_data[filtered_data['Availability'] == availability_filter]
    table.delete(*table.get_children())  # clear the table
    for i, row in filtered_data.iterrows():
        table.insert('', 'end', values=row.tolist(), iid=i)

# Create the GUI
root = tk.Tk()
root.title("Course Availability")

table_frame = ttk.Frame(root)
table_frame.pack(padx=10, pady=10)

table = ttk.Treeview(table_frame, columns=['Code', 'Department', 'Status', 'Quota', 'Current Quota', 'Availability'])
table.heading('#0', text='Index')
table.heading('Code', text='Code')
table.heading('Department', text='Department')
table.heading('Status', text='Status')
table.heading('Quota', text='Quota')
table.heading('Current Quota', text='Current Quota')
table.heading('Availability', text='Availability')
table.column('#0', width=0, stretch=tk.NO)
table.column('Code', width=120, anchor=tk.CENTER)
table.column('Department', width=100, anchor=tk.CENTER)
table.column('Status', width=100, anchor=tk.CENTER)
table.column('Quota', width=100, anchor=tk.CENTER)
table.column('Current Quota', width=120, anchor=tk.CENTER)
table.column('Availability', width=120, anchor=tk.CENTER)
table.pack()

code_filter_var = tk.StringVar()
code_filter_var.set('')
code_filter_label = ttk.Label(root, text='Filter by Code:')
code_filter_label.pack()
code_filter_dropdown = ttk.Combobox(root, textvariable=code_filter_var, values=testingdata['Code'].str.extract(r'([A-Z]+)').dropna().drop_duplicates().values.squeeze().tolist() + [''], state='readonly')
code_filter_dropdown.pack()

availability_filter_var = tk.StringVar()
availability_filter_var.set('')
availability_filter_label = ttk.Label(root, text='Filter by Availability:')
availability_filter_label.pack()
availability_filter_dropdown = ttk.Combobox(root, textvariable=availability_filter_var, values=['Full', 'Available', 'Try to get consent', ''], state='readonly')
availability_filter_dropdown.pack()

update_table_button = ttk.Button(root, text="Update Table", command=lambda: update_table(code_filter_var.get(), availability_filter_var.get()))
update_table_button.pack(padx=10, pady=10)

# Insert data into the table
for i, row in testingdata.iterrows():
    table.insert('', 'end', values=row.tolist(), iid=i)

root.mainloop()


In [40]:
testdata

,Code,Department,Status,Quota,Current Quota,Availability
0,DSAI301.01,ALL,ALL,Consent Of Instructor,0,Available
1,ASIA502.02,ALL,ALL,Consent Of Instructor,2,Available
2,ASIA519.02,ALL,ALL,Consent Of Instructor,5,Available
3,ASIA521.02,ALL,ALL,Consent Of Instructor,1,Available
4,ASIA527.02,ALL,ALL,Consent Of Instructor,2,Available
...,...,...,...,...,...,...
3062,FA 49F.01,ALL,ALL,Consent Of Instructor,39,Available
3063,FA 49S.01,ALL,ALL,Consent Of Instructor,16,Available
3064,LIT 311.01,ALL,ALL,Consent Of Instructor,0,Available
3065,LIT 311.01,ALL,UNDERGRADUATE,40,65,Try to get consent


In [42]:
'''
testdata.to_excel("testing6.xlsx", index=False)
'''